Set up all columns before any fetching is done
set up 'root's (rankings, server file)
For every iteration, it should fetch all unknowns, store their "wave", then extract all unseen players from that cohort
Then once no unseen players are there, store as CSV file

Factor out everything into a new repo

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from base64 import b64encode as b64
import asyncio
import time
import ast
import itertools


In [2]:
from username_to_user_id import username_to_user_id # from original repo
old_names = set(username_to_user_id.keys())
print("old names:  ", len(old_names))

ratings_url = "https://acquire.tlstyer.com/stats/data/ratings.json"
resp = requests.get(ratings_url)
stats = json.loads(resp.text)

rated_names = set()
for key in stats.keys():
    for rating in stats[key]:
        name = rating[0]
        rated_names.add(name)
print("rated names:  ", len(rated_names))


player_page_root = "https://acquire.tlstyer.com/stats/data/users/" # /encoded_username.json
# extract all names from JSON response

old names:   23331
rated names:   726


In [3]:
def encode_username(s_ascii = "username"):
    s_code = b64(s_ascii.encode("utf-8")).decode("utf-8")
    s_code = s_code.replace("=", "")
    s_code = s_code.replace("+", "")
    s_code = s_code.replace("\/", "_")
    return s_code




In [4]:
print(len(list(rated_names) + list(old_names)))
print(len(rated_names.union(old_names)))
print(len(rated_names.intersection(old_names)))

24057
23694
363


In [5]:
players = pd.DataFrame()
usernames = pd.Series(list(rated_names.union(old_names)))
players['username'] = usernames
players['encoded_username'] = players['username'].apply(encode_username)
players['url'] = player_page_root + players['encoded_username'] + ".json"
players

,username,encoded_username,url
0,scott.hausman,c2NvdHQuaGF1c21hbg,https://acquire.tlstyer.com/stats/data/users/c...
1,CR winner,Q1Igd2lubmVy,https://acquire.tlstyer.com/stats/data/users/Q...
2,radar1086,cmFkYXIxMDg2,https://acquire.tlstyer.com/stats/data/users/c...
3,H.R.Puffn'stuff,SC5SLlB1ZmZuJ3N0dWZm,https://acquire.tlstyer.com/stats/data/users/S...
4,Master 01,TWFzdGVyIDAx,https://acquire.tlstyer.com/stats/data/users/T...
...,...,...,...
23689,1234,MTIzNA,https://acquire.tlstyer.com/stats/data/users/M...
23690,Xabo,WGFibw,https://acquire.tlstyer.com/stats/data/users/W...
23691,v-locity,di1sb2NpdHk,https://acquire.tlstyer.com/stats/data/users/d...
23692,Tim!!,VGltISE,https://acquire.tlstyer.com/stats/data/users/V...


In [17]:
players['find_degree'] = np.nan

flavors = ['Singles2', 'Singles3', 'Singles4', 'Teams']
attrs = ['rating', 'rate_moe', 'games_count', 'record', 'last_played']

words = list(itertools.product(flavors, attrs))
cols = ["_".join(two) for two in words]

for col in cols:
    players[f'{col}'] = np.nan

players.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23694 entries, 0 to 23693
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   username              23694 non-null  object 
 1   encoded_username      23694 non-null  object 
 2   url                   23694 non-null  object 
 3   find_degree           23694 non-null  int64  
 4   Singles2_games_count  0 non-null      float64
 5   Singles2_rating       0 non-null      float64
 6   Singles2_rate_moe     0 non-null      float64
 7   Singles2_record       0 non-null      float64
 8   Singles2_last_played  0 non-null      float64
 9   Singles3_rating       0 non-null      float64
 10  Singles3_rate_moe     0 non-null      float64
 11  Singles3_games_count  0 non-null      float64
 12  Singles3_record       0 non-null      float64
 13  Singles3_last_played  0 non-null      float64
 14  Singles4_rating       0 non-null      float64
 15  Singles4_rate_moe  

In [18]:
players.to_pickle("./players.pkl")

In [21]:
players = pd.read_pickle("./players.pkl")
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23694 entries, 0 to 23693
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   username              23694 non-null  object 
 1   encoded_username      23694 non-null  object 
 2   url                   23694 non-null  object 
 3   find_degree           23694 non-null  int64  
 4   Singles2_games_count  0 non-null      float64
 5   Singles2_rating       0 non-null      float64
 6   Singles2_rate_moe     0 non-null      float64
 7   Singles2_record       0 non-null      float64
 8   Singles2_last_played  0 non-null      float64
 9   Singles3_rating       0 non-null      float64
 10  Singles3_rate_moe     0 non-null      float64
 11  Singles3_games_count  0 non-null      float64
 12  Singles3_record       0 non-null      float64
 13  Singles3_last_played  0 non-null      float64
 14  Singles4_rating       0 non-null      float64
 15  Singles4_rate_moe  

In [22]:
len(players[players['stats_page'] == ""])

9681

In [23]:
players['stats_page'] = players['stats_page'].str.decode("utf-8")

In [24]:
sum(players['stats_page'].isna())

9681

In [25]:
opponents = set()

In [27]:
sum(players['username'] == "test1")  # known username from server file

1

In [28]:
known_players = set(players['username'])
new_players = set()

In [381]:
valid_users = players[~players['stats_page'].isna()]

#Extract ratings:
valid_user = valid_users.sample()
stats_object = json.loads(valid_user['stats_page'].values[0])
ratings = stats_object['ratings']
    # return ratings

games = stats_object['games']
print("")
print(len(games))

#Extract Games Opponents:
for game in games:
    match_player = (game[2][0][0])
    if match_player not in known_players:
        if match_player not in new_players:
            new_players.add(match_player)
    # return new_players
print(len(new_players))


11
96


In [382]:
print(new_players)

{'YumYum', 'hoff2021', 'Jkr4', 'Domkop', 'focus2021', 'xiaopiting', 'eero', 'baroquester', 'Thales College Student', 'zigzag', 'just play fast', 'zeevr-BTW', 'spookygirl101', 'Tom Wambsgans', 'Shaggy', 'Sid the Kid', 'aaron2021', 'disenchanted 66.237', 'gogogadget', 'Bruno', 'Hilary', 'Nurse', 'prettykitty', 'Tune', '2300skidoo', 'Ron DeSantis', 'space672', 'Cleveland Indians', 'cupcake', 'Juice Box', 'ahmed.hany.abdelwahab@gmail.com', 'trent025', 'train7878', 'kurtweis22', 'stewie', 'space21', 'gmoney', 'LeopardGeckoFan', 'Skibaba2121', 'nevermind', 'Frank', 'Used2bA', 'piruli22', 'Wolvo', 'LSU', 'janssentan96', 'alwayswinning', 'Buddy6985', 'businessman2021', 'billyboy34', 'empireofthemind', 'DrSolano', 'H-Rah2022', 'Kali', '##', 'Alias2021', 'A.A.Ron', 'Doc Holliday', 'Spaceeer', 'Jakob-dec', 'dwomo', 'Fahrver21', 'itsamegary', 'Cousin Greg', 'dist347', 'cosmocougar!', 'tbow', 'Ruffy', 'GaryTx', 'mrblac', '2021plsbbttr', 'IAdoug2021', 'magenta', 'sllubad2003', 'Crown13', 'muditb', '